In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.vision import *

path = untar_data(URLs.MNIST_TINY)

# the problem with the error

In [2]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path=path,train='train', test='valid',
                                  valid_pct=0.2, ds_tfms=tfms, size=26)

data.train_ds, data.valid_ds

(DatasetTfm(ImageClassificationDataset of len 568),
 DatasetTfm(ImageClassificationDataset of len 141))

First sign of something wrong. The test set is not loaded but I have specified it above.

In [3]:
data.test_ds

AssertionError: You didn't specify a test set for this DataBunch.

This is not a problem when training the model, but then I get an error when getting the predictions on the test dataset

In [4]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit(2)

Total time: 00:13
epoch  train_loss  valid_loss  accuracy
1      0.606474    0.489490    0.737589  (00:06)
2      0.459560    0.312031    0.858156  (00:06)



In [5]:
res = learn.get_preds(is_test=True)

TypeError: object of type 'NoneType' has no len()

This is the problem, my solution is below

# Testing the solution

To make thing easy to test, I have overriden the from_folder method.

In [7]:
class ImageDataBunch2(ImageDataBunch):
    @classmethod
    def from_folder(cls, path:PathOrStr, train:PathOrStr='train', valid:PathOrStr='valid',
                    test:Optional[PathOrStr]=None, valid_pct=None, **kwargs:Any)->'ImageDataBunch':
        "Create from imagenet style dataset in `path` with `train`,`valid`,`test` subfolders (or provide `valid_pct`)."
        path=Path(path)
        if valid_pct is None:
            train_ds = ImageClassificationDataset.from_folder(path/train)
            datasets = [train_ds, ImageClassificationDataset.from_folder(path/valid, classes=train_ds.classes)]
        else: datasets = ImageClassificationDataset.from_folder(path/train, valid_pct=valid_pct)

        if test: datasets.append(ImageClassificationDataset.from_folder(
            path/test,classes=datasets[0].classes))

        return cls.create(*datasets, path=path, **kwargs)

In [8]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch2.from_folder(path=path,train='train', test='valid',
                                  valid_pct=0.2, ds_tfms=tfms, size=26)

data.train_ds, data.valid_ds

(DatasetTfm(ImageClassificationDataset of len 558),
 DatasetTfm(ImageClassificationDataset of len 151))

Now I get something that makes sense:

In [9]:
data.test_ds

DatasetTfm(ImageClassificationDataset of len 699)

Now I can train and test on the test dataset

In [10]:
learn = create_cnn(data, models.resnet18, metrics=accuracy)
learn.fit(2)

Total time: 00:19
epoch  train_loss  valid_loss  accuracy
1      0.529082    0.283582    0.913907  (00:07)
2      0.440459    0.382659    0.774834  (00:11)



In [11]:
r1, r2 = learn.get_preds(is_test=True)
len(r1), len(r2)

(699, 699)